In [28]:
import pandas as pd
import numpy as np
import time
import re
import json
import tqdm
import glob
import requests as rq # http requests
import bs4 as bs4 # beautiful soup for parsing

# 1.0 - Data Collect

In [54]:
df = pd.read_json('./parsed_videos.json', lines=True)

In [55]:
## Get all unique links
unique_links = df.link.unique()
print("Lenght unique links:",len(unique_links))
print("Number of duplicated links", len(df)-len(unique_links))

Lenght unique links: 1780
Number of duplicated links 32


In [56]:
url = 'http://www.youtube.com{link}'

In [ ]:
for link in unique_links:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)", link).group(1)
    with open("./raw_data/video_{}.html".format(link_name),'w+') as output:
        output.write(response.text)
    time.sleep(2)

# 2.0 - Data Processing

In [63]:
with open("parsed_video_info.json","w+") as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data/video*"))):
        with open(video_file,"r+") as inp:
            page_html = inp.read() 
            parsed = bs4.BeautifulSoup(page_html,"html.parser") 
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")}) 
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")}) 
            channel = parsed.find_all("a",attrs={"href":re.compile(r"channel")}) 
            meta = parsed.find_all("meta") 
        
            
            data = dict()
            
            for e in class_watch:
                colname ="_".join(e['class'])
                if 'clearfix' in colname: ## Jump condition to clear fix to not create one column per video
                    continue
                data[colname] = e.text.strip()
                
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
                    
            for link_num, e in enumerate(channel):
                data['channel_link_{}'.format(link_num)] = e['href']
            
            
            output.write("{}\n".format(json.dumps(data)))
                    
                    

# 3.0 - Verification

In [64]:
df = pd.read_json('./parsed_video_info.json',lines=True)
df.shape

(1780, 173)

In [65]:
df.columns

Index(['content-alignment_watch-small', 'watch-playlist_player-height',
       'watch-queue-header', 'watch-queue-info', 'watch-queue-info-icon',
       'watch-queue-title', 'watch-queue-control-bar_control-bar-button',
       'watch-queue-mole-info', 'watch-queue-control-bar-icon',
       'watch-queue-icon_yt-sprite',
       ...
       'channel_link_13', 'channel_link_14', 'channel_link_15',
       'channel_link_16', 'channel_link_17', 'channel_link_18',
       'channel_link_19', 'channel_link_20', 'channel_link_21',
       'channel_link_22'],
      dtype='object', length=173)

In [66]:
pd.set_option(display.c)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch7-speedyg-area,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,channel_link_1,channel_link_2,watch-view-count,watch8-sentiment-actions,watch7-views-info,watch-channel-brand-div,watch-channel-brand-div-text,watch-sidebar-live-chat,channel_link_3,channel_link_4,channel_link_5,channel_link_6,channel_link_7,channel_link_8,channel_link_9,channel_link_10,channel_link_11,channel_link_12,watch-meta-item_has-image,watch-sidebar-discussion,watch-meta-item,yt-pl-watch-queue-overlay,watch-skeleton,watch_history,watch_later,watch_related_mix,what_to_watch,ytd-watch-card-album-list-renderer,ytd-watch-card-collage-renderer,ytd-watch-card-single-image-renderer,ytd-watch-card-video-list-renderer,ytd-generic-watch-card,watch-card-header,watch-card-title,watch-card-labels,ytd-artist-watch-card-renderer,ytd-sho